<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/Computer%20Vision%20Algorithms%20parg%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Histogram of oriented gradients

In [1]:
pip install opencv-python scikit-image


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure

def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Image not found at the path: {image_path}")
    return image

def compute_hog(image):
    hog_features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), block_norm='L2-Hys',
                                  visualize=True, multichannel=False)
    return hog_features, hog_image

def display_images(original_image, hog_image):
    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(original_image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title('HOG Image')
    plt.imshow(hog_image_rescaled, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

def main():
    image_path = 'path_to_your_image.jpg'  # Replace with your image path
    image = load_image(image_path)

    hog_features, hog_image = compute_hog(image)

    print(f"HOG feature vector length: {len(hog_features)}")

    display_images(image, hog_image)

if __name__ == "__main__":
    main()


Region based convolutional neural networks

In [ ]:
pip install torch torchvision matplotlib


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)


In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A',
    'dining table', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = T.Compose([T.ToTensor()])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

def detect_objects(model, image, threshold=0.5):
    with torch.no_grad():
        predictions = model(image)

    pred_classes = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in predictions[0]['labels'].numpy()]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in predictions[0]['boxes'].detach().numpy()]
    pred_scores = predictions[0]['scores'].detach().numpy()

    pred_t = [pred_scores.index(x) for x in pred_scores if x > threshold][-1]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_classes = pred_classes[:pred_t+1]

    return pred_boxes, pred_classes

def draw_boxes(image_path, boxes, pred_classes):
    image = cv2.imread(image_path)
    for i, box in enumerate(boxes):
        cv2.rectangle(image, box[0], box[1], color=(0, 255, 0), thickness=2)
        cv2.putText(image, pred_classes[i], box[0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, lineType=cv2.LINE_AA)

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

def main():
    image_path = 'path_to_your_image.jpg'  # Replace with your image path
    image = load_image(image_path)

    boxes, pred_classes = detect_objects(model, image, threshold=0.5)

    draw_boxes(image_path, boxes, pred_classes)

if __name__ == "__main__":
    main()


YOLO(you only look once)

In [3]:
pip install opencv-python numpy


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load YOLO
def load_yolo():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return net, classes, output_layers

# Load image
def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    return img, height, width, channels

def detect_objects(img, net, output_layers):
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    return outs

def get_bounding_boxes(outs, height, width, threshold=0.5):
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    return boxes, confidences, class_ids

# Draw bounding boxes
def draw_bounding_boxes(img, boxes, confidences, class_ids, classes):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 10), font, 1, color, 2)
    return img

# Display image
def display_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

# Main function
def main(image_path):
    net, classes, output_layers = load_yolo()
    img, height, width, channels = load_image(image_path)
    outs = detect_objects(img, net, output_layers)
    boxes, confidences, class_ids = get_bounding_boxes(outs, height, width)
    img = draw_bounding_boxes(img, boxes, confidences, class_ids, classes)
    display_image(img)

if __name__ == "__main__":
    image_path = "path_to_your_image.jpg"  # Replace with your image path
    main(image_path)


 SSD(single shot multibox detector)

In [4]:
pip install torch torchvision opencv-python


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

model = torchvision.models.detection.ssd300_vgg16(pretrained=True)
model.eval()  # Set the model to evaluation mode

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A',
    'dining table', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor()
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

def detect_objects(model, image, threshold=0.5):
    with torch.no_grad():
        predictions = model(image)[0]

    pred_classes = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in predictions['labels'].numpy()]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in predictions['boxes'].detach().numpy()]
    pred_scores = predictions['scores'].detach().numpy()

    filtered_boxes = []
    filtered_classes = []
    for i, score in enumerate(pred_scores):
        if score > threshold:
            filtered_boxes.append(pred_boxes[i])
            filtered_classes.append(pred_classes[i])

    return filtered_boxes, filtered_classes

def draw_boxes(image_path, boxes, pred_classes):
    image = cv2.imread(image_path)
    for i, box in enumerate(boxes):
        cv2.rectangle(image, box[0], box[1], color=(0, 255, 0), thickness=2)
        cv2.putText(image, pred_classes[i], box[0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, lineType=cv2.LINE_AA)

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

def main():
    image_path = 'path_to_your_image.jpg'  # Replace with your image path
    image = load_image(image_path)

    boxes, pred_classes = detect_objects(model, image, threshold=0.5)

    draw_boxes(image_path, boxes, pred_classes)

if __name__ == "__main__":
    main()


Fast R-**CNN**

In [5]:
pip install torch torchvision opencv-python


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A',
    'dining table', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

def detect_objects(model, image, threshold=0.5):
    with torch.no_grad():
        predictions = model(image)

    pred_classes = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in predictions[0]['labels'].numpy()]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in predictions[0]['boxes'].detach().numpy()]
    pred_scores = predictions[0]['scores'].detach().numpy()

    filtered_boxes = []
    filtered_classes = []
    for i, score in enumerate(pred_scores):
        if score > threshold:
            filtered_boxes.append(pred_boxes[i])
            filtered_classes.append(pred_classes[i])

    return filtered_boxes, filtered_classes

def draw_boxes(image_path, boxes, pred_classes):
    image = cv2.imread(image_path)
    for i, box in enumerate(boxes):
        cv2.rectangle(image, box[0], box[1], color=(0, 255, 0), thickness=2)
        cv2.putText(image, pred_classes[i], box[0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, lineType=cv2.LINE_AA)

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

def main():
    image_path = 'path_to_your_image.jpg'  # Replace with your image path
    image = load_image(image_path)

    boxes, pred_classes = detect_objects(model, image, threshold=0.5)

    draw_boxes(image_path, boxes, pred_classes)

if __name__ == "__main__":
    main()


Faster R-CNN


In [6]:
pip install torch torchvision


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = torchvision.datasets.CocoDetection(root='path/to/coco', annFile='path/to/annotations', transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)
model = fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 91  # 80 classes of COCO + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Training loop
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {losses.item()}")
model.eval()
with torch.no_grad():
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        outputs = model(images)
